<a href="https://colab.research.google.com/github/ekqlsrla/ESAA/blob/main/PROJECT/%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/Data/movies_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Data/movies_test.csv')
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [ ]:
# 결측치가 많은 데이터 제거
train = train.drop(['dir_prev_bfnum'],axis = 1)
test =  test.drop(['dir_prev_bfnum'],axis = 1)

In [ ]:
# 감독명 : 너무 다양해서 제거
train = train.drop(['director'],axis = 1)
test = test.drop(['director'],axis = 1)

In [ ]:
# 제목 : 의미가 없기 때문에 제거
train = train.drop(['title'],axis= 1)
test = test.drop(['title'],axis= 1)

In [ ]:
train.distributor.value_counts()

CJ 엔터테인먼트        54
롯데엔터테인먼트         52
(주)NEW           30
(주)마운틴픽쳐스        29
(주)쇼박스           26
                 ..
OAL(올)            1
(주)에이원 엔터테인먼트     1
(주)콘텐츠 윙          1
위더스필름             1
퍼스트런              1
Name: distributor, Length: 169, dtype: int64

In [ ]:
# 상위 5개의 배급사를 제외하고 '기타'로처리
distributor_list = train.distributor.value_counts()[:5]
def func(distributor):
    if distributor in distributor_list:
        return distributor
    else:
        return '기타'

train['distributor'] = train['distributor'].apply(lambda x : func(x))
test['distributor'] = test['distributor'].apply(lambda x : func(x))

In [ ]:
# 개봉일을 바탕으로 년,월 변수 생성
train['년'] = train['release_time'].apply(lambda x: int(x[:4]))
train['월'] = train['release_time'].apply(lambda x: int(x[5:7]))
train =  train.drop(['release_time'],axis = 1)

test['년'] = test['release_time'].apply(lambda x: int(x[:4]))
test['월'] = test['release_time'].apply(lambda x: int(x[5:7]))
test =  test.drop(['release_time'],axis = 1)

In [ ]:
# 원핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
#정규화
y_train = train['box_off_num']
X_train = train.drop('box_off_num', axis=1)
X_test = test

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# MinMaxScaler
minmax_scaler = preprocessing.MinMaxScaler()
norm_fit = minmax_scaler.fit(X_train)
X_train_norm = norm_fit.transform(X_train)
X_test_norm = norm_fit.transform(X_test)

# standardization
standard_scaler = preprocessing.StandardScaler()
stan_fit = standard_scaler.fit(X_train)
X_train_stan = stan_fit.transform(X_train)
X_test_stan = stan_fit.transform(X_test)

# 로그 변환
import numpy as np
X_train_log = np.log1p(X_train)
X_test_log = np.log1p(X_test)

## 모델정의 및 학습

In [ ]:
train_x = train.drop(['box_off_num'],axis= 1)
train_y = train['box_off_num']

In [ ]:
model=RandomForestRegressor(n_estimators=100)
model.fit(train_x,train_y)

NameError: ignored

---
## 1.스태킹 앙상블 

- 점수 : 1265442.23121


In [ ]:
#라이브러리 로딩

import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge,Lasso

* 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

# param_grid of GridSearchCV
params = {
    'n_estimators':[500,1000],
    'max_depth' : [4, 6, 8, 10], 
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

# RandomForest 모델 학습
rf_clf = RandomForestRegressor(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1, scoring='neg_mean_squared_error', verbose=1)
grid_cv.fit(X_train, y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)

In [ ]:
params = {
    'n_estimators' : [500,1000],
    'learning_rate' : [0.01,0.05,0.1]
}

gbm_clf = GradientBoostingRegressor(random_state = 0)
grid_cv = GridSearchCV(gbm_clf, param_grid = params,cv = 2, verbose = 1,scoring = 'neg_mean_squared_error')
grid_cv.fit(X_train,y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)

In [ ]:
params = {
    'max_depth' : [4, 6, 8, 10],
    'n_estimators':[100,500,1000],
    'learing_rate' : [0.01,0.05,0.01]
}

xgb = xgb.XGBRegressor(random_state = 0)
grid_cv = GridSearchCV(xgb,param_grid = params, cv = 2, verbose = 1, scoring = 'neg_mean_squared_error')
grid_cv.fit(X_train,y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)

In [ ]:
params = {
    'num_leaves' : [30,40,50,60],
    'n_estimators':[100,500,1000],
    'learing_rate' : [0.01,0.05,0.01],
    'max_depth' : [2,3,4,5]
}

lgbm = lgb.LGBMRegressor(random_state = 0)
grid_cv = GridSearchCV(lgbm,param_grid = params, cv = 2, verbose = 1, scoring = 'neg_mean_squared_error')
grid_cv.fit(X_train,y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)

## 2. GBM

### 1) 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

params = {'n_estimators' : [100,500], 'learning_rate' : [0.05,0.1]}

gbm = GradientBoostingRegressor(random_state=0)
grid_cv = GridSearchCV(gbm, param_grid=params, cv=2, verbose=1, scoring='neg_mean_squared_error')
grid_cv.fit(X_train, y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)

### 2) MinMax

점수: 1265380.591066857	

In [ ]:
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05)
gbm.fit(X_train_norm, y_train)
gbm_pred = gbm.predict(X_test_norm)

### 3) Standardization

점수: 1351207.2462183223	

In [ ]:
gbm.fit(X_train_stan, y_train)
gbm_pred = gbm.predict(X_test_stan)

### 4) 로그 변환

점수: 1267526.093198045	

In [ ]:
gbm.fit(X_train_log, y_train)
gbm_pred = gbm.predict(X_test_log)

## 3. XGBoost

### 1) 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

params = {'max_depth':[4, 6, 8, 10], 'n_estimators':[100,500,1000], 'learing_rate':[0.01,0.05,0.1]}


xgb = xgb.XGBRegressor()

grid_cv = GridSearchCV(xgb, param_grid=params, cv=2, verbose=1, scoring='neg_mean_squared_error')
grid_cv.fit(X_train, y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)

### 2) MinMax

점수: 1444906.3206120685	

In [ ]:
# MinMaxScaler

xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.01, max_depth=4)

xgb.fit(X_train_norm, y_train)
xgb_pred = xgb.predict(X_test_norm)

### 3) Standardization

점수: 1444933.6663520725	

In [ ]:
xgb.fit(X_train_stan, y_train)
xgb_pred = xgb.predict(X_test_stan)

### 4) 로그 변환



점수: 1444933.6663520725   

In [ ]:
xgb.fit(X_train_log, y_train)
xgb_pred = xgb.predict(X_test_log)

## 3. RandomForest

### 1) 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[80,100,500],
    'max_depth' : [4,6,8,10],
    'min_samples_leaf' : [4,6,8,12],
    'min_samples_split' : [4,6,8,16]
}

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train_log, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도:{0:.4f}'.format(grid_cv.best_score_))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


In [ ]:
rf_clf = RandomForestClassifier()

### 2) MinMax

점수: 2103496.5955934785	

In [ ]:
rf_clf.fit(X_train_norm, y_train)
rf_clf_pred_norm = lgbm.predict(X_test_norm)

### 3) Standardization

점수: 2119014.0894160094	

In [ ]:
rf_clf.fit(X_train_stan, y_train)
rf_clf_pred_stan = lgbm.predict(X_test_stan)

### 4) 로그 변환

점수: 2098766.3752528387	

In [ ]:
rf_clf.fit(X_train_log, y_train)
rf_clf_pred_log = lgbm.predict(X_test_log)

## LightGBM

### 1) 하이퍼 파라미터 최적화

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators=100,random_state=156)

params = {
    'num_leaves' : [8,16],
    'max_depth' : [-1,1,5,10],
    'min_child_samples':[5,8,10,20],
    'subsample':[0.5,0.8],
    'learning_rate' : [0.01,0.05, 0.1]
}

grid_cv = GridSearchCV(lgbm, param_grid=params, cv=2, verbose = 1, scoring = 'neg_mean_squared_error')
grid_cv.fit(X_train_norm,y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)

In [ ]:
lgbm = LGBMClassifier()

### 2) MinMax

점수: 2054027.285027303	

In [ ]:
lgbm.fit(X_train_norm, y_train)
lgbm_pred_norm = lgbm.predict(X_test_norm)

### 3) Standardization

점수: 	1868506.59590569	

In [ ]:
lgbm.fit(X_train_stan, y_train)
lgbm_pred_stan = lgbm.predict(X_test_stan)

### 4) 로그 변환

점수: 	1761140.0603350203	

In [ ]:
lgbm.fit(X_train_log, y_train)
lgbm_pred_log = lgbm.predict(X_test_log)

## 5. 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings 

warnings.filterwarnings("ignore")

In [ ]:
lr = LogisticRegression()

### 1) 하이퍼 파라미터 최적화

In [ ]:
params = {'penalty' : ['l2','l1'],
         'C' : [0.01, 0.1, 1, 1, 5, 10]}

grid_clf = GridSearchCV(lr, param_grid = params, scoring='accuracy', cv=2)
grid_clf.fit(X_train, y_train)
print('최적의 하이퍼 파라미터 : \n', grid_clf.best_params_)
print('최고 예측 정확도 : ', grid_clf.best_score_)

In [ ]:
lr = LogisticRegression(penalty = 'l2', C=5)

### 2) MinMax

- 점수 : 1855805.38578

In [ ]:
lr.fit(X_train_norm, y_train)
lr_pred_norm = lr.predict(X_test_norm)

### 3) Standardization

- 점수 : 2191414.823729781

In [ ]:
lr.fit(X_train_stan, y_train)
lr_pred_stan = lr.predict(X_test_stan)

### 4) 로그 변환

- 점수 : 2401377.5015082355	

In [ ]:
lr.fit(X_train_log, y_train)
lr_pred_log = lr.predict(X_test_log)

## 학습 된 모델로 예측 데이터 생성

In [ ]:
pred = model.predict(test)

## 제출파일 생성

In [ ]:
submission = pd.read_csv('/content/submission.csv')
submission

In [ ]:
submission['box_off_num'] = pred

In [ ]:
submission

In [ ]:
submission.to_csv('베이스라인.csv',index = False)

In [ ]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots(1,1)
fig.suptitle('Model Comparison')

barwidth = 0.5

ax1.bar(0, 1265442.23121, width=barwidth, edgecolor='white') # 스태킹 앙상블
ax1.bar(1, 1265380.591066857, width=barwidth, edgecolor='white') # GBM - minmax 
ax1.bar(2, 1351207.2462183223, width=barwidth, edgecolor='white') # GBM - stan
ax1.bar(3, 1267526.093198045, width=barwidth, edgecolor='white') # GBM - log
ax1.bar(4, 1444906.3206120685, width=barwidth, edgecolor='white') # XGB - minmax
ax1.bar(5, 1444933.6663520725, width=barwidth, edgecolor='white') # XGB - stan
ax1.bar(6, 1444933.6663520725, width=barwidth, edgecolor='white') # XGB - log
ax1.bar(7, 2103496.59559347851, width=barwidth, edgecolor='white') # RandomForest - minmax
ax1.bar(8, 2119014.0894160094, width=barwidth, edgecolor='white') # RandomForest - stan
ax1.bar(9, 2098766.3752528387, width=barwidth, edgecolor='white') # RandomForest - log
ax1.bar(10, 2054027.285027303, width=barwidth, edgecolor='white') # LGBM - minmax
ax1.bar(11, 1868506.59590569, width=barwidth, edgecolor='white') # LGBM- stan
ax1.bar(12, 1761140.0603350203, width=barwidth, edgecolor='white') # LGBM - log
ax1.bar(13, 1855805.38578, width=barwidth, edgecolor='white') # Logisitc - minmax
ax1.bar(14, 2191414.823729781, width=barwidth, edgecolor='white') # Logistic- stan
ax1.bar(15, 2401377.5015082355, width=barwidth, edgecolor='white') # Logistic - log

ax1.set_xlabel('Classification Model')
labels = ['','ensemble', 'GBM minmax', 'GBM stan', 'GBM log', 
          'XGB minmax', 'XGB stan', 'XGB log',
         'RF minmax', 'RF stan', 'RF log',
         'LGBM minmax', 'LGBM stan', 'LGBM log',
         'LR minmax', 'LR stan', 'LR log']
ax1.set_xticklabels(labels, rotation=45)
ax1.set_ylabel('RMSE')
plt.show()

GBM(minmax)의 RMSE 값이 가장 낮게 나타났기 때문에 좋은 예측 성능을 보인다고 할 수 있음
